In [18]:
#Import the required lib
import os
import numpy as np
import pandas as pd 
from utils import pct_change, preprocess_and_split_df, convert_to_dataset, data_preparation, plot_data, create_bollinger_limits
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import random
import torch
import statistics
from LSTM_model_future_change import LSTM_FutureChangeGeneral, LSTM_FutureChange

import warnings
warnings.filterwarnings('ignore')

In [19]:
# Upload the testing and validation dataframes 
# create the data dict again 
CRYPTO_DICT = { 
            "ADA": {}, 
            "BCH": {},
            "BNB": {}, 
            "BTC": {},
            "ETH": {},
            "LTC": {}, 
            "NEO": {},
            "TRX": {},
            "XRP": {},
}

# upload the data 

for Crypto_code in CRYPTO_DICT:
    CRYPTO_DICT[Crypto_code]["df_val"]  = pd.read_pickle(f"Training_Data/Data_preprocessed/{Crypto_code}/{Crypto_code}-val_df.pkl")
    CRYPTO_DICT[Crypto_code]["df_test"]  = pd.read_pickle(f"Training_Data/Data_preprocessed/{Crypto_code}/{Crypto_code}-test_df.pkl")

In [31]:
CRYPTO_DICT["BTC"]["df_test"].head(10)

,unix,date,symbol,close,mid_price_1,mid_price_2,Volume USDT,SMA,EMA0,EMA1,EMA2,RSI,upper,lower,upper_check,lower_check,future_price,future_change
318944,1627707240,07/31/2021,BTC/USDT,1.211071,0.411546,0.684028,1.007001,0.428165,0.055027,0.626576,-0.042810,63.706992,0.839225,-0.266744,1,0,0.733889,1
318945,1627707300,07/31/2021,BTC/USDT,-0.050905,0.655188,0.856290,0.111526,0.189359,0.653417,0.597509,-0.285091,68.632921,0.529305,-0.266289,1,0,0.050129,1
318946,1627707360,07/31/2021,BTC/USDT,-0.205117,0.072938,-0.202684,1.100856,0.241162,0.370447,-0.239198,-0.273788,80.731364,0.491564,-0.168277,1,0,0.338247,1
318947,1627707420,07/31/2021,BTC/USDT,0.722975,0.225426,0.415833,0.689553,0.458191,0.474338,-0.205766,-0.343913,85.517834,0.945770,-0.330310,1,0,0.214703,1
318948,1627707480,07/31/2021,BTC/USDT,0.049170,0.751145,0.558432,4.054216,0.215221,0.421083,-0.443098,-0.763588,80.938914,0.622177,-0.322061,1,0,-0.959201,0
318949,1627707540,07/31/2021,BTC/USDT,0.333093,0.179732,0.265005,0.854757,0.440625,-0.605043,-0.370522,-0.470865,80.753855,0.990046,-0.393327,1,0,-0.051418,0
318952,1627707720,07/31/2021,BTC/USDT,-0.050900,0.078174,-0.796926,-0.432073,0.959720,-0.333718,-0.456956,-0.290869,62.342857,-0.331753,1.473648,1,0,-0.213473,0
318953,1627707780,07/31/2021,BTC/USDT,-0.421503,-0.687587,-0.262134,0.499403,0.902873,-0.347416,-0.216342,-0.621145,57.816640,-1.066893,2.092402,1,0,-0.831539,0
318954,1627707840,07/31/2021,BTC/USDT,-0.077981,0.251767,-0.379138,0.401749,0.478776,-0.880303,0.390724,-0.427946,59.196961,-0.292661,0.853075,1,0,0.237134,0
318955,1627707900,07/31/2021,BTC/USDT,-0.210595,-0.617676,-0.210678,-0.100251,0.326235,-0.237419,-0.286380,-0.721248,57.030842,-0.245213,0.623735,0,1,0.157387,0


In [27]:
# Test on BTC
df_val = CRYPTO_DICT["BTC"]["df_val"].reset_index(drop=True)
df_test = CRYPTO_DICT["BTC"]["df_test"].reset_index(drop=True)
df_test.head(4)

,unix,date,symbol,close,mid_price_1,mid_price_2,Volume USDT,SMA,EMA0,EMA1,EMA2,RSI,upper,lower,upper_check,lower_check,future_price,future_change
0,1627707240,07/31/2021,BTC/USDT,1.211071,0.411546,0.684028,1.007001,0.428165,0.055027,0.626576,-0.042810,63.706992,0.839225,-0.266744,1,0,0.733889,1
1,1627707300,07/31/2021,BTC/USDT,-0.050905,0.655188,0.856290,0.111526,0.189359,0.653417,0.597509,-0.285091,68.632921,0.529305,-0.266289,1,0,0.050129,1
2,1627707360,07/31/2021,BTC/USDT,-0.205117,0.072938,-0.202684,1.100856,0.241162,0.370447,-0.239198,-0.273788,80.731364,0.491564,-0.168277,1,0,0.338247,1
3,1627707420,07/31/2021,BTC/USDT,0.722975,0.225426,0.415833,0.689553,0.458191,0.474338,-0.205766,-0.343913,85.517834,0.945770,-0.330310,1,0,0.214703,1


In [28]:
# upload the model 
# Upload the general model to be compared by each coin-specific model 
MAIN_MODEL = LSTM_FutureChangeGeneral(13, 160, 2, 0.4, 1).float()
MAIN_MODEL.load_state_dict(torch.load("Trained-models/LSTM-model/General-LSTM_model.pt", map_location='cpu'))

<All keys matched successfully>

In [29]:
df_test.iloc[:60]

,unix,date,symbol,close,mid_price_1,mid_price_2,Volume USDT,SMA,EMA0,EMA1,EMA2,RSI,upper,lower,upper_check,lower_check,future_price,future_change
0,1627707240,07/31/2021,BTC/USDT,1.211071,0.411546,0.684028,1.007001,0.428165,0.055027,0.626576,-0.042810,63.706992,0.839225,-0.266744,1,0,0.733889,1
1,1627707300,07/31/2021,BTC/USDT,-0.050905,0.655188,0.856290,0.111526,0.189359,0.653417,0.597509,-0.285091,68.632921,0.529305,-0.266289,1,0,0.050129,1
2,1627707360,07/31/2021,BTC/USDT,-0.205117,0.072938,-0.202684,1.100856,0.241162,0.370447,-0.239198,-0.273788,80.731364,0.491564,-0.168277,1,0,0.338247,1
3,1627707420,07/31/2021,BTC/USDT,0.722975,0.225426,0.415833,0.689553,0.458191,0.474338,-0.205766,-0.343913,85.517834,0.945770,-0.330310,1,0,0.214703,1
4,1627707480,07/31/2021,BTC/USDT,0.049170,0.751145,0.558432,4.054216,0.215221,0.421083,-0.443098,-0.763588,80.938914,0.622177,-0.322061,1,0,-0.959201,0
5,1627707540,07/31/2021,BTC/USDT,0.333093,0.179732,0.265005,0.854757,0.440625,-0.605043,-0.370522,-0.470865,80.753855,0.990046,-0.393327,1,0,-0.051418,0
6,1627707720,07/31/2021,BTC/USDT,-0.050900,0.078174,-0.796926,-0.432073,0.959720,-0.333718,-0.456956,-0.290869,62.342857,-0.331753,1.473648,1,0,-0.213473,0
7,1627707780,07/31/2021,BTC/USDT,-0.421503,-0.687587,-0.262134,0.499403,0.902873,-0.347416,-0.216342,-0.621145,57.816640,-1.066893,2.092402,1,0,-0.831539,0
8,1627707840,07/31/2021,BTC/USDT,-0.077981,0.251767,-0.379138,0.401749,0.478776,-0.880303,0.390724,-0.427946,59.196961,-0.292661,0.853075,1,0,0.237134,0
9,1627707900,07/31/2021,BTC/USDT,-0.210595,-0.617676,-0.210678,-0.100251,0.326235,-0.237419,-0.286380,-0.721248,57.030842,-0.245213,0.623735,0,1,0.157387,0
